In [1]:
import pandas as pd

file_hen = "CALERA HEN PRICES.xlsx"
file_wholesale = "CALERA WHOLESALE HISTORIC PRICE.xlsx"
file_midagri = "Midagri Price.xlsx"
file_farm = "FARM PRICES.xlsx"
file_academic = "ACADEMIC PERIOD DATES.xlsx"


hen = pd.read_excel(file_hen)
wholesale = pd.read_excel(file_wholesale)
midagri = pd.read_excel(file_midagri)
farm = pd.read_excel(file_farm)
academic = pd.read_excel(file_academic)

In [2]:
farm.rename(columns={"FRAM PRICE x KG": "FARM PRICE x KG"}, inplace=True)

midagri.rename(columns={"DATE": "Date"}, inplace=True)
farm.rename(columns={"DATE": "Date"}, inplace=True)

hen["Date"] = pd.to_datetime(hen["Date"])
wholesale["Date"] = pd.to_datetime(wholesale["Date"])
midagri["Date"] = pd.to_datetime(midagri["Date"])
farm["Date"] = pd.to_datetime(farm["Date"])

In [3]:
hen_avg_price = hen.groupby("Date")["PRICE WITH TAXES x KG"].mean().reset_index()
print(hen_avg_price.head())

        Date  PRICE WITH TAXES x KG
0 2021-01-02               7.230090
1 2021-01-03               7.186410
2 2021-01-04               7.004858
3 2021-01-05               7.254631
4 2021-01-06               7.207804


In [4]:
midagri_avg_price = midagri.groupby("Date")["PRICE WITH TAXES x KG"].mean().reset_index()
print(midagri_avg_price.head())

        Date  PRICE WITH TAXES x KG
0 2021-01-04                   3.75
1 2021-01-05                   3.71
2 2021-01-06                   3.75
3 2021-01-07                   3.78
4 2021-01-08                   3.86


In [5]:
farm_avg_price = farm.groupby("Date")["FARM PRICE x KG"].mean().reset_index()
print(farm_avg_price.head())

        Date  FARM PRICE x KG
0 2021-01-02         3.349720
1 2021-01-16         4.200000
2 2021-01-26         4.367143
3 2021-01-27         4.467143
4 2021-01-28         4.588097


In [6]:
hen_avg_price = hen_avg_price.rename(columns={"PRICE WITH TAXES x KG": "AVG HEN PRICE WITH TAXES x KG"})
midagri_avg_price = midagri_avg_price.rename(columns={"PRICE WITH TAXES x KG": "AVG MIDAGRI PRICE WITH TAXES x KG"})
farm_avg_price = farm_avg_price.rename(columns={"FARM PRICE x KG": "AVG FARM PRICE WITH TAXES x KG"})
# 以 Date 为键合并两张表
wholesale_hen = wholesale.merge(hen_avg_price, on="Date", how="left")
wholesale_hen_midagri = wholesale_hen.merge(midagri_avg_price, on="Date", how="left")
wholesale_mergeall = wholesale_hen_midagri.merge(farm_avg_price, on="Date", how="left")

print(wholesale_mergeall.head())

        Date ZONE    CHANNEL SALES ZONE  PRICE WITH NO TAXES x KG  \
0 2021-01-02  Sur  Mayorista   Arequipa                  3.051674   
1 2021-01-03  Sur  Mayorista   Arequipa                  3.120002   
2 2021-01-04  Sur  Mayorista   Arequipa                  3.038903   
3 2021-01-05  Sur  Mayorista   Arequipa                  3.518057   
4 2021-01-06  Sur  Mayorista   Arequipa                  3.039112   

   PRICE WITH TAXES x KG         KG      SOLES  AVG HEN PRICE WITH TAXES x KG  \
0               3.600975  19587.610   59775.00                       7.230090   
1               3.681602  22559.280   70385.00                       7.186410   
2               3.585906  16864.766   51250.39                       7.004858   
3               4.151307  55234.620  194318.55                       7.254631   
4               3.586152  32649.540   99225.61                       7.207804   

   AVG MIDAGRI PRICE WITH TAXES x KG  AVG FARM PRICE WITH TAXES x KG  
0                          

In [7]:
wholesale_mergeall.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12543 entries, 0 to 12542
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Date                               12543 non-null  datetime64[ns]
 1   ZONE                               12543 non-null  object        
 2   CHANNEL                            12543 non-null  object        
 3   SALES ZONE                         12543 non-null  object        
 4   PRICE WITH NO TAXES x KG           12543 non-null  float64       
 5   PRICE WITH TAXES x KG              12543 non-null  float64       
 6   KG                                 12543 non-null  float64       
 7   SOLES                              12543 non-null  float64       
 8   AVG HEN PRICE WITH TAXES x KG      12530 non-null  float64       
 9   AVG MIDAGRI PRICE WITH TAXES x KG  11374 non-null  float64       
 10  AVG FARM PRICE WITH TAXES x KG    

In [8]:
print("hen:\n", wholesale_mergeall.isnull().sum())

hen:
 Date                                    0
ZONE                                    0
CHANNEL                                 0
SALES ZONE                              0
PRICE WITH NO TAXES x KG                0
PRICE WITH TAXES x KG                   0
KG                                      0
SOLES                                   0
AVG HEN PRICE WITH TAXES x KG          13
AVG MIDAGRI PRICE WITH TAXES x KG    1169
AVG FARM PRICE WITH TAXES x KG        399
dtype: int64


In [9]:
wholesale_mergeall.fillna(wholesale_mergeall.mean(numeric_only=True), inplace=True)
print(wholesale_mergeall.isnull().sum())  

Date                                 0
ZONE                                 0
CHANNEL                              0
SALES ZONE                           0
PRICE WITH NO TAXES x KG             0
PRICE WITH TAXES x KG                0
KG                                   0
SOLES                                0
AVG HEN PRICE WITH TAXES x KG        0
AVG MIDAGRI PRICE WITH TAXES x KG    0
AVG FARM PRICE WITH TAXES x KG       0
dtype: int64


In [10]:
import numpy as np

for col in wholesale_mergeall.columns:
    if wholesale_mergeall[col].dtype in ["object", "datetime64[ns]"]:
        continue

    inf_rows = wholesale_mergeall[np.isinf(wholesale_mergeall[col])]
    
    if not inf_rows.empty:
        print(f"column '{col}' has inf:\n", inf_rows)

In [11]:
wholesale_mergeall["market_avg_price"] = wholesale_mergeall[["PRICE WITH TAXES x KG", "AVG FARM PRICE WITH TAXES x KG"]].mean(axis=1, skipna=True)
print(wholesale_mergeall.head())

        Date ZONE    CHANNEL SALES ZONE  PRICE WITH NO TAXES x KG  \
0 2021-01-02  Sur  Mayorista   Arequipa                  3.051674   
1 2021-01-03  Sur  Mayorista   Arequipa                  3.120002   
2 2021-01-04  Sur  Mayorista   Arequipa                  3.038903   
3 2021-01-05  Sur  Mayorista   Arequipa                  3.518057   
4 2021-01-06  Sur  Mayorista   Arequipa                  3.039112   

   PRICE WITH TAXES x KG         KG      SOLES  AVG HEN PRICE WITH TAXES x KG  \
0               3.600975  19587.610   59775.00                       7.230090   
1               3.681602  22559.280   70385.00                       7.186410   
2               3.585906  16864.766   51250.39                       7.004858   
3               4.151307  55234.620  194318.55                       7.254631   
4               3.586152  32649.540   99225.61                       7.207804   

   AVG MIDAGRI PRICE WITH TAXES x KG  AVG FARM PRICE WITH TAXES x KG  \
0                         

In [12]:
wholesale_mergeall["delta"] = wholesale_mergeall["market_avg_price"] - wholesale_mergeall["AVG MIDAGRI PRICE WITH TAXES x KG"]
print(wholesale_mergeall.head())

        Date ZONE    CHANNEL SALES ZONE  PRICE WITH NO TAXES x KG  \
0 2021-01-02  Sur  Mayorista   Arequipa                  3.051674   
1 2021-01-03  Sur  Mayorista   Arequipa                  3.120002   
2 2021-01-04  Sur  Mayorista   Arequipa                  3.038903   
3 2021-01-05  Sur  Mayorista   Arequipa                  3.518057   
4 2021-01-06  Sur  Mayorista   Arequipa                  3.039112   

   PRICE WITH TAXES x KG         KG      SOLES  AVG HEN PRICE WITH TAXES x KG  \
0               3.600975  19587.610   59775.00                       7.230090   
1               3.681602  22559.280   70385.00                       7.186410   
2               3.585906  16864.766   51250.39                       7.004858   
3               4.151307  55234.620  194318.55                       7.254631   
4               3.586152  32649.540   99225.61                       7.207804   

   AVG MIDAGRI PRICE WITH TAXES x KG  AVG FARM PRICE WITH TAXES x KG  \
0                         

In [13]:
wholesale_mergeall

,Date,ZONE,CHANNEL,SALES ZONE,PRICE WITH NO TAXES x KG,PRICE WITH TAXES x KG,KG,SOLES,AVG HEN PRICE WITH TAXES x KG,AVG MIDAGRI PRICE WITH TAXES x KG,AVG FARM PRICE WITH TAXES x KG,market_avg_price,delta
0,2021-01-02,Sur,Mayorista,Arequipa,3.051674,3.600975,19587.610,59775.00,7.230090,6.957344,3.349720,3.475348,-3.481997
1,2021-01-03,Sur,Mayorista,Arequipa,3.120002,3.681602,22559.280,70385.00,7.186410,6.957344,6.250357,4.965980,-1.991365
2,2021-01-04,Sur,Mayorista,Arequipa,3.038903,3.585906,16864.766,51250.39,7.004858,3.750000,6.250357,4.918131,1.168131
3,2021-01-05,Sur,Mayorista,Arequipa,3.518057,4.151307,55234.620,194318.55,7.254631,3.710000,6.250357,5.200832,1.490832
4,2021-01-06,Sur,Mayorista,Arequipa,3.039112,3.586152,32649.540,99225.61,7.207804,3.750000,6.250357,4.918255,1.168255
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12538,2025-01-18,Norte,Mayorista,Trujillo,7.203390,8.500000,1534.000,11050.00,8.738389,5.830000,5.438110,6.969055,1.139055
12539,2025-01-20,Norte,Mayorista,Trujillo,4.344276,5.126245,19411.330,84328.17,8.752663,6.372143,5.390055,5.258150,-1.113993
12540,2025-01-21,Norte,Mayorista,Trujillo,4.618633,5.449987,251.228,1160.33,8.511955,5.850000,5.195330,5.322659,-0.527341
12541,2025-01-22,Norte,Mayorista,Trujillo,4.576270,5.399999,209.544,958.93,8.580004,6.356111,5.123670,5.261835,-1.094277


In [14]:
wholesale_mergeall.rename(columns={"PRICE WITH TAXES x KG": "wholesale_unit_price"}, inplace=True)
wholesale_mergeall.rename(columns={"AVG HEN PRICE WITH TAXES x KG": "hen_unit_price"}, inplace=True)
wholesale_mergeall.rename(columns={"AVG MIDAGRI PRICE WITH TAXES x KG": "midagri_unit_price"}, inplace=True)
wholesale_mergeall.rename(columns={"AVG FARM PRICE WITH TAXES x KG": "farm_unit_price"}, inplace=True)

In [15]:
wholesale_mergeall

,Date,ZONE,CHANNEL,SALES ZONE,PRICE WITH NO TAXES x KG,wholesale_unit_price,KG,SOLES,hen_unit_price,midagri_unit_price,farm_unit_price,market_avg_price,delta
0,2021-01-02,Sur,Mayorista,Arequipa,3.051674,3.600975,19587.610,59775.00,7.230090,6.957344,3.349720,3.475348,-3.481997
1,2021-01-03,Sur,Mayorista,Arequipa,3.120002,3.681602,22559.280,70385.00,7.186410,6.957344,6.250357,4.965980,-1.991365
2,2021-01-04,Sur,Mayorista,Arequipa,3.038903,3.585906,16864.766,51250.39,7.004858,3.750000,6.250357,4.918131,1.168131
3,2021-01-05,Sur,Mayorista,Arequipa,3.518057,4.151307,55234.620,194318.55,7.254631,3.710000,6.250357,5.200832,1.490832
4,2021-01-06,Sur,Mayorista,Arequipa,3.039112,3.586152,32649.540,99225.61,7.207804,3.750000,6.250357,4.918255,1.168255
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12538,2025-01-18,Norte,Mayorista,Trujillo,7.203390,8.500000,1534.000,11050.00,8.738389,5.830000,5.438110,6.969055,1.139055
12539,2025-01-20,Norte,Mayorista,Trujillo,4.344276,5.126245,19411.330,84328.17,8.752663,6.372143,5.390055,5.258150,-1.113993
12540,2025-01-21,Norte,Mayorista,Trujillo,4.618633,5.449987,251.228,1160.33,8.511955,5.850000,5.195330,5.322659,-0.527341
12541,2025-01-22,Norte,Mayorista,Trujillo,4.576270,5.399999,209.544,958.93,8.580004,6.356111,5.123670,5.261835,-1.094277


In [16]:
file_path = "merged.xlsx"
wholesale_mergeall.to_excel(file_path, index=False)
print(f"Success to {file_path}")

Success to merged.xlsx
